In [8]:
from mat_ceng import get_column_area_loads
import shapely
from shapely import (
    Point, 
    MultiPoint, 
    Polygon, 
    MultiPolygon, 
    LineString, 
    MultiLineString,
    GeometryCollection
)
import json
import numpy as np

In [9]:
with open('floor_data.json', 'r') as file:
    floor_data = json.load(file)

with open('occupancy_loading.json', 'r') as file:
    occupancy_loading = json.load(file)

display(occupancy_loading)
display(floor_data)

{'heavy_occupancy': {'dead': 6.0, 'live': 4.8, 'wind': 1.0},
 'light_occupancy': {'dead': 5.1, 'live': 2.4, 'wind': 1.0}}

{'slab_outline': [[4500.0, 0.0],
  [0.0, 0.0],
  [0.0, 3000.0],
  [1000.0, 3000.0],
  [1000.0, 5500.0],
  [0.0, 5500.0],
  [0.0, 15000.0],
  [20000.0, 15000.0],
  [20000.0, 0.0],
  [16000.0, -2500.0],
  [12500.0, 0.0],
  [7500.0, -1000.0],
  [4500.0, -1000.0]],
 'slab_openings': [[[12500.0, 11000.0],
   [18500.0, 11000.0],
   [18500.0, 13000.0],
   [12500.0, 13000.0]],
  [[11500.0, 4000.0],
   [14500.0, 4000.0],
   [14500.0, 5000.0],
   [11500.0, 5000.0]]],
 'light_occupancy': [[10500.0, 15000.0],
  [10500.0, -400.0000000000002],
  [7500.0, -1000.0],
  [4500.0, -1000.0],
  [4500.0, 0.0],
  [0.0, 0.0],
  [0.0, 3000.0],
  [1000.0, 3000.0],
  [1000.0, 5500.0],
  [0.0, 5500.0],
  [0.0, 15000.0]],
 'heavy_occupancy': [[10500.0, 15000.0],
  [10500.0, -400.0000000000002],
  [12500.0, 0.0],
  [16000.0, -2500.0],
  [20000.0, 0.0],
  [20000.0, 15000.0]],
 'walls': [[[8300.0, 7000.0],
   [8300.0, 9900.0],
   [4500.0, 9900.0],
   [4500.0, 7000.0],
   [4900.0, 7000.0],
   [4900.0, 9500.0],
   [7900

In [10]:

col_area_load = get_column_area_loads(floor_data,occupancy_loading,300)


In [11]:
mp = MultiPolygon([Polygon([(0,0),(1,1),(0,1)])])
mp.geom_type


'MultiPolygon'

In [12]:
import plotly.graph_objects as go
# x,y = zip(*floor_data['slab_outline'])
# fig = go.Figure(go.Scatter(x=x, y=y, fill="toself",opacity =0.5,hoverinfo='skip'))
# x_col, y_col = list(x)+[None],list(y)+[None]

x_edge,y_edge = zip(*floor_data['slab_outline'])

x_col, y_col = [],[]
for col in col_area_load:
    x,y = col.column_outline.exterior.xy
    x_col = x_col + list(x) + [None]
    y_col = y_col + list(y) + [None]

fig = go.Figure(go.Scatter(x=x_col, y=y_col, fill="toself",opacity =0.5,hoverinfo='skip'))

x_col, y_col = [],[]
for col in col_area_load:
    if col.trib_area.geom_type == 'Polygon':
        x,y = col.trib_area.exterior.xy
        x_col = x_col + list(x) + [None]
        y_col = y_col + list(y) + [None]
    elif col.trib_area.geom_type == 'MultiPolygon':
        for plg in col.trib_area.geoms:
            x,y = plg.exterior.xy
            x_col = x_col + list(x) + [None]
            y_col = y_col + list(y) + [None]

fig.add_trace(go.Scatter(
    x=x_col,
    y=y_col,
    fill="toself",
    opacity =0.25,
    hoverinfo='skip'
))

fig.add_trace(go.Scatter(
    x=x_edge,
    y=y_edge,
    mode="lines",
    hoverinfo='skip'
))

x = [col.column_outline.centroid.x for col in col_area_load]
y = [col.column_outline.centroid.y for col in col_area_load]
labels = [round(col.get_combined_load(np.array([1,1,0])),2) for col in col_area_load]
trib_areas = [str(round(col.trib_area.area / 1e6,2)) + ' m²' for col in col_area_load]
# Create scatter trace of text labels
fig.add_trace(
    go.Scatter(
        x=x, y=y, mode="markers", 
        opacity = 0.25,
        hoverinfo='skip',
        showlegend=False,
        marker_size=[float(area[:-3])*1.5 for area in trib_areas],
        marker={"color": "green"}
))


fig.add_trace(go.Scatter(
    x=x,
    y=y,
    text=labels,
    mode="text",
    hovertext = trib_areas,
    hoverinfo="text"
))

fig.update_layout(
    width = 900,
    height = 900,
    title = "Column Design Forces [KN]"
)

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

